In [ ]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark.sql.functions as f

In [ ]:
@dlt.table(
    comment="Source Data"
)
def dlt_source_table():
    return (spark.table('iowa.iowa_delta'))

@dlt.table(
    comment="Corrected data"
)
@dlt.expect("City", "City <> 'WINDSOR HEIGHTS'")
@dlt.expect_or_drop("ID", "ID <= 5")
@dlt.expect_or_fail("Date", "Date IS NOT NULL")
def dlt_iowa_corrected():
    return (
        dlt.read("dlt_source_table").withColumn("Sale_Dollars", f.col("Sale_Dollars") + 200)
    )
    
@dlt.table(
    comment="Aggregated data"
)
def dlt_iowa_aggregated():
    return(
        dlt.read("dlt_iowa_corrected")
        .filter(f.col("County").isNotNull() & f.col("County").isin("Lyon", "Iowa")) \
        .groupBy("County").agg(f.round(f.sum("Sale_Dollars"), 2).alias("Sale_Dollars"))
    )

In [ ]:
#@dlt.expect_all_or_drop({"City": "City = "Los Angeles","ID": "ID IS NOT NULL AND County IS NOT NULL"})